In [73]:
import lseg.data as ld
ld.open_session(
    app_key="ff25e00926b145099c655332d969c9762ed6ff7b"
)

<lseg.data.session.Definition object at 0x11e654a50 {name='workspace'}>

In [74]:
indices = ['.BFX','.OMXC20 ', '.OMXH25','.SBF120','.GDAXI','.AEX','.AMX','.OBX','.OMXS30']
date = ['2019-12-31', '2020-12-31', '2021-12-31', '2022-12-31', '2023-12-31', '2024-12-31']


In [83]:
def load_all_index_data(indices, dates):
    """
    Load index constituent data for each index for ALL dates that work.
    
    Parameters:
    -----------
    indices : list
        List of index tickers
    dates : list
        List of dates to fetch data for
    
    Returns:
    --------
    dict : Dictionary with index as key and list of (dataframe, date) tuples as value
    """
    results = {}
    failed_combinations = []
    
    for index in indices:
        results[index] = []
        print(f"\n{'='*50}")
        print(f"Loading data for {index}")
        print(f"{'='*50}")
        
        for date in dates:
            try:
                print(f"Trying to load {index} with date {date}...")
                df = ld.get_data(
                    universe=[index],
                    fields=[          
                        f'TR.IndexConstituentRIC(SDate={date})',
                        f'TR.IndexConstituentName(SDate={date})'
                    ] 
                )
                
                # Check if we got valid data
                if df is not None and not df.empty:
                    # Add date column to track which date was used
                    df['Date'] = date
                    df['Index'] = index
                    results[index].append((df, date))
                    print(f"✓ Successfully loaded {index} with date {date} ({len(df)} constituents)")
                else:
                    print(f"✗ No data returned for {index} with date {date}")
                    failed_combinations.append((index, date))
                    
            except Exception as e:
                print(f"✗ Error loading {index} with date {date}: {str(e)}")
                failed_combinations.append((index, date))
                continue
        
        if not results[index]:
            print(f"⚠ No data loaded for {index} with any date")
        else:
            print(f"✓ Successfully loaded {len(results[index])} dates for {index}")
    
    # Summary
    print(f"\n{'='*60}")
    print(f"SUMMARY")
    print(f"{'='*60}")
    for index in indices:
        successful_dates = len(results[index])
        print(f"{index}: {successful_dates}/{len(dates)} dates loaded")
    
    if failed_combinations:
        print(f"\n⚠ {len(failed_combinations)} failed combinations (out of {len(indices) * len(dates)} total)")
    print(f"{'='*60}\n")
    
    return results

# Load data for all indices and all dates
all_data = load_all_index_data(indices, date)


Loading data for .BFX
Trying to load .BFX with date 2019-12-31...
✓ Successfully loaded .BFX with date 2019-12-31 (20 constituents)
Trying to load .BFX with date 2020-12-31...
✓ Successfully loaded .BFX with date 2020-12-31 (20 constituents)
Trying to load .BFX with date 2021-12-31...
✓ Successfully loaded .BFX with date 2021-12-31 (20 constituents)
Trying to load .BFX with date 2022-12-31...
✓ Successfully loaded .BFX with date 2022-12-31 (20 constituents)
Trying to load .BFX with date 2023-12-31...
✓ Successfully loaded .BFX with date 2023-12-31 (20 constituents)
Trying to load .BFX with date 2024-12-31...
✓ Successfully loaded .BFX with date 2024-12-31 (20 constituents)
✓ Successfully loaded 6 dates for .BFX

Loading data for .OMXC20 
Trying to load .OMXC20  with date 2019-12-31...
✓ Successfully loaded .OMXC20  with date 2019-12-31 (20 constituents)
Trying to load .OMXC20  with date 2020-12-31...
✓ Successfully loaded .OMXC20  with date 2020-12-31 (20 constituents)
Trying to load 

An error occurred while requesting URL('http://localhost:9030/api/udf').
	ReadTimeout('timed out')


✗ Error loading .OMXS30 with date 2021-12-31: timed out
Trying to load .OMXS30 with date 2022-12-31...
✓ Successfully loaded .OMXS30 with date 2022-12-31 (29 constituents)
Trying to load .OMXS30 with date 2023-12-31...
✓ Successfully loaded .OMXS30 with date 2023-12-31 (30 constituents)
Trying to load .OMXS30 with date 2024-12-31...
✓ Successfully loaded .OMXS30 with date 2024-12-31 (30 constituents)
✓ Successfully loaded 4 dates for .OMXS30

SUMMARY
.BFX: 6/6 dates loaded
.OMXC20 : 6/6 dates loaded
.OMXH25: 6/6 dates loaded
.SBF120: 6/6 dates loaded
.GDAXI: 6/6 dates loaded
.AEX: 6/6 dates loaded
.AMX: 6/6 dates loaded
.OBX: 6/6 dates loaded
.OMXS30: 4/6 dates loaded

⚠ 2 failed combinations (out of 54 total)



In [84]:
# Access individual index data
# Example: Get all data for a specific index
# for df, date_used in all_data['.BFX']:
#     print(f"Data for .BFX on {date_used}: {len(df)} constituents")
#     display(df.head())

# Combine all dataframes into one
import pandas as pd

if all_data:
    all_dfs = []
    for index, data_list in all_data.items():
        for df, date_used in data_list:
            all_dfs.append(df)
    
    if all_dfs:
        combined_df = pd.concat(all_dfs, ignore_index=True)
        print(f"Combined dataframe shape: {combined_df.shape}")
        print(f"Date range: {combined_df['Date'].min()} to {combined_df['Date'].max()}")
        print(f"Unique indices: {combined_df['Index'].nunique()}")
        print(f"Unique dates: {combined_df['Date'].nunique()}")
        display(combined_df.head(10))
    else:
        print("No data was loaded successfully")
combined_df.to_csv('all_data.csv', index=False)

Combined dataframe shape: (1913, 5)
Date range: 2019-12-31 to 2024-12-31
Unique indices: 9
Unique dates: 6


,Instrument,Constituent RIC,Constituent Name,Date,Index
0,.BFX,SOLB.BR,SOLVAY ORD,2019-12-31,.BFX
1,.BFX,ACKB.BR,ACKERMANS V.HAAR ORD,2019-12-31,.BFX
2,.BFX,ARGX.BR,ARGEN X NV,2019-12-31,.BFX
3,.BFX,PROX.BR,PROXIMUS,2019-12-31,.BFX
4,.BFX,INGA.AS,ING GROEP ORD,2019-12-31,.BFX
5,.BFX,ONTEX.BR,ONTEX GRP NV,2019-12-31,.BFX
6,.BFX,APAM.AS,APERAM ORD,2019-12-31,.BFX
7,.BFX,TNET.BR^J23,TELENET GRP HLDG ORD,2019-12-31,.BFX
8,.BFX,WDPP.BR,WDP-SICAFI REIT,2019-12-31,.BFX
9,.BFX,GBLB.BR,GROUPE BRUXELLES LAMBERT ORD,2019-12-31,.BFX


In [85]:
combined_df

,Instrument,Constituent RIC,Constituent Name,Date,Index
0,.BFX,SOLB.BR,SOLVAY ORD,2019-12-31,.BFX
1,.BFX,ACKB.BR,ACKERMANS V.HAAR ORD,2019-12-31,.BFX
2,.BFX,ARGX.BR,ARGEN X NV,2019-12-31,.BFX
3,.BFX,PROX.BR,PROXIMUS,2019-12-31,.BFX
4,.BFX,INGA.AS,ING GROEP ORD,2019-12-31,.BFX
...,...,...,...,...,...
1908,.OMXS30,SHBa.ST,SVENSKA HANDELSBANKEN CLS A ORD,2024-12-31,.OMXS30
1909,.OMXS30,VOLVb.ST,VOLVO CL B ORD,2024-12-31,.OMXS30
1910,.OMXS30,KINVb.ST,KINNEVIK STK B ORD,2024-12-31,.OMXS30
1911,.OMXS30,ABB.ST,ABB LTD ORD,2024-12-31,.OMXS30


In [92]:
dates = ['2020-12-31', '2021-12-31']
omsx30_data = {}
for date in dates:
    omsx30_data[date] = ld.get_data(
        universe = ['.OMXS30'],
        fields = [          
            f'TR.IndexConstituentRIC(SDate={date})',
            f'TR.IndexConstituentName(SDate={date})'
        ] 
    )
omsx30 = pd.concat(list(omsx30_data.values()))
# Add OMXS30 data to combined_df
omsx30['Date'] = pd.to_datetime(dates[0])  # Using first date as default
omsx30['Index'] = '.OMXS30'
omsx30 = omsx30.rename(columns={
    'Constituent RIC': 'Constituent RIC',
    'Constituent Name': 'Constituent Name',
})
omsx30 = omsx30[['Instrument', 'Constituent RIC', 'Constituent Name', 'Date', 'Index']]
combined_df = pd.concat([combined_df, omsx30], ignore_index=True)
combined_df

,Instrument,Constituent RIC,Constituent Name,Date,Index
0,.BFX,SOLB.BR,SOLVAY ORD,2019-12-31,.BFX
1,.BFX,ACKB.BR,ACKERMANS V.HAAR ORD,2019-12-31,.BFX
2,.BFX,ARGX.BR,ARGEN X NV,2019-12-31,.BFX
3,.BFX,PROX.BR,PROXIMUS,2019-12-31,.BFX
4,.BFX,INGA.AS,ING GROEP ORD,2019-12-31,.BFX
...,...,...,...,...,...
1968,.OMXS30,SHBa.ST,SVENSKA HANDELSBANKEN CLS A ORD,2020-12-31 00:00:00,.OMXS30
1969,.OMXS30,VOLVb.ST,VOLVO CL B ORD,2020-12-31 00:00:00,.OMXS30
1970,.OMXS30,KINVb.ST,KINNEVIK STK B ORD,2020-12-31 00:00:00,.OMXS30
1971,.OMXS30,ABB.ST,ABB ORD,2020-12-31 00:00:00,.OMXS30


In [93]:
combined_df.to_csv('all_data.csv', index=False)